# LSTM Learning with TensorFlow

We will demonstrate fitting a recurrent neural network to a text file.

Based on https://github.com/skyer9/hello_lstm, ported to Jupyter notebook.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.rnn import BasicLSTMCell, GRUCell
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.seq2seq import sequence_loss

import numpy as np

In [2]:
#X_train = "Hello, World!"
# X_train = "안녕하세요, World!"
X_train = ("if you want to build a ship, don't drum up people together to "
           "collect wood and don't assign them tasks and work, but rather "
           "teach them to long for the endless immensity of the sea.")

#nb_epoches = 25
nb_epoches = 100
batch_size = 10
num_steps = 200 # Length of gradient unchaining

In [3]:
with open(r'd:\projects\gltestplus\src\gltestplus.cpp', encoding='utf-8') as f:
    text = f.read()
    #text = f.readlines()

X_train = text#[:1000].replace('\r', '').replace('\n', '')
print(len(X_train))

84019


In [4]:
def create_vocabulary(sequence):
    vocab = {}
    for i in range(len(sequence)):
        ch = sequence[i]
        if ch in vocab:
            vocab[ch] += 1
        else:
            vocab[ch] = 1
    vocab_rev = sorted(vocab, key=vocab.get, reverse=True)
    vocab = dict([(x, y) for (y, x) in enumerate(vocab_rev)])
    return vocab, vocab_rev


def sentence_to_token_ids(sentence, vocabulary):
    characters = [sentence[i:i+1] for i in range(0, len(sentence), 1)]
    return [vocabulary.get(w) for w in characters]


def token_ids_to_one_hot(token_ids, num_classes=10):
    token_ids_one_hot = np.zeros((len(token_ids), num_classes))
    token_ids_one_hot[np.arange(len(token_ids)), token_ids] = 1
    return token_ids_one_hot



In [5]:
sequence_length = len(X_train) - 1

X_train_vocab, X_train_vocab_rev = create_vocabulary(X_train)
hidden_size = len(X_train_vocab)
num_classes = len(X_train_vocab)

X_train_ids = sentence_to_token_ids(X_train, X_train_vocab)
X_data = X_train_ids[:-1]
Y_data = X_train_ids[1:]
X_data_one_hot = [token_ids_to_one_hot(X_data, num_classes)]
Y_data = [Y_data]



In [6]:
used = False

with tf.name_scope('input_X'):
    X = tf.placeholder(tf.float32, [None, num_steps, hidden_size], name='X')

with tf.name_scope('output_Y'):
    Y = tf.placeholder(tf.int32, [None, num_steps], name='Y')

with tf.name_scope('lstm'):
    #cell = BasicLSTMCell(num_units=hidden_size)
    cell = GRUCell(num_units=hidden_size)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell,
                                         X,
                                         initial_state=initial_state,
                                         dtype=tf.float32)
    tf.summary.histogram('rnn_outputs', outputs)

    X_for_fc = tf.reshape(outputs, [-1, hidden_size])
    outputs = fully_connected(inputs=X_for_fc,
                              num_outputs=num_classes,
                              activation_fn=None)

    outputs = tf.reshape(outputs, [batch_size, num_steps, num_classes])
    tf.summary.histogram('outputs', outputs)

    weights = tf.ones([batch_size, num_steps])
    tf.summary.histogram('weights', weights)
with tf.name_scope('sequence_loss'):
    sloss = sequence_loss(logits=outputs,
                                  targets=Y,
                                  weights=weights)
    tf.summary.scalar('sequence_loss', sloss)
with tf.name_scope('loss'):
    loss = tf.reduce_mean(sloss)
    tf.summary.scalar('loss', loss)
with tf.name_scope('train'):
    train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

with tf.name_scope('prediction'):
    prediction = tf.argmax(outputs, axis=2)
    
used = True
print(cell)

In [7]:
print(X_data_one_hot[0].shape, len(Y_data[0]), Y)

(84018, 94) 84018 Tensor("output_Y/Y:0", shape=(?, 200), dtype=int32)


In [11]:
GRUCell?

In [10]:
tf.strided_slice?

In [15]:
tf.nn.dynamic_rnn?

In [19]:
tf.nn.l2_loss?

In [8]:
tf.trainable_variables()

[<tf.Variable 'rnn/gru_cell/gates/kernel:0' shape=(188, 188) dtype=float32_ref>,
 <tf.Variable 'rnn/gru_cell/gates/bias:0' shape=(188,) dtype=float32_ref>,
 <tf.Variable 'rnn/gru_cell/candidate/kernel:0' shape=(188, 94) dtype=float32_ref>,
 <tf.Variable 'rnn/gru_cell/candidate/bias:0' shape=(94,) dtype=float32_ref>,
 <tf.Variable 'fully_connected/weights:0' shape=(94, 94) dtype=float32_ref>,
 <tf.Variable 'fully_connected/biases:0' shape=(94,) dtype=float32_ref>]

In [10]:
sequence_loss?

In [9]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [10]:
def rnn_minibatch_sequencer(raw_data, batch_size, sequence_size, nb_epochs):
    """
    Divides the data into batches of sequences so that all the sequences in one batch
    continue in the next batch. This is a generator that will keep returning batches
    until the input data has been seen nb_epochs times. Sequences are continued even
    between epochs, apart from one, the one corresponding to the end of raw_data.
    The remainder at the end of raw_data that does not fit in an full batch is ignored.
    :param raw_data: the training text
    :param batch_size: the size of a training minibatch
    :param sequence_size: the unroll size of the RNN
    :param nb_epochs: number of epochs to train on
    :return:
        x: one batch of training sequences
        y: on batch of target sequences, i.e. training sequences shifted by 1
        epoch: the current epoch number (starting at 0)
    """
    data = np.array(raw_data)
    data_len = data.shape[0]
    # using (data_len-1) because we must provide for the sequence shifted by 1 too
    nb_batches = (data_len - 1) // (batch_size * sequence_size)
    assert nb_batches > 0, "Not enough data, even for a single batch. Try using a smaller batch_size."
    rounded_data_len = nb_batches * batch_size * sequence_size
    xdata = np.reshape(data[0:rounded_data_len], [batch_size, nb_batches * sequence_size])
    ydata = np.reshape(data[1:rounded_data_len + 1], [batch_size, nb_batches * sequence_size])

    for epoch in range(nb_epochs):
        for batch in range(nb_batches):
            x = xdata[:, batch * sequence_size:(batch + 1) * sequence_size]
            y = ydata[:, batch * sequence_size:(batch + 1) * sequence_size]
            x = np.roll(x, -epoch, axis=0)  # to continue the text from epoch to epoch (do not reset rnn state!)
            y = np.roll(y, -epoch, axis=0)
            yield x, y, epoch, batch, nb_batches


In [10]:
fileWriter = tf.summary.FileWriter('TensorFlowLSTM/16')
fileWriter.add_graph(sess.graph)

In [11]:
merged_summary = tf.summary.merge_all()

In [ ]:
# Initialize the training variables
sess.run(tf.global_variables_initializer())
printed_epoch = 0
printed_percentage = -1
losses_in_batches = []

In [15]:
# From datalab vm1
for x, y_, i, batch, nb_batches in rnn_minibatch_sequencer(X_data, batch_size, num_steps, nb_epochs=nb_epoches):
#for i in range(nb_epoches):
    X_data_one_hot = [token_ids_to_one_hot(x, num_classes) for x in x]
    Y_data = y_
    #        print('X: ', X_data_one_hot, ', Y: ', y_.shape)
    l, _ = sess.run([loss, train], feed_dict={X: X_data_one_hot, Y: Y_data})
    result = sess.run(prediction, feed_dict={X: X_data_one_hot})
    s = sess.run(merged_summary, feed_dict={X: X_data_one_hot, Y: Y_data})
    fileWriter.add_summary(s, i)
    if printed_epoch < i:
        print(i, "loss:", np.mean(losses_in_batches)) #, "prediction: ", result, "true Y: ", Y_data)
        losses_in_batches = []
        printed_percentage = -1
    losses_in_batches.append(l)
    if printed_percentage < batch * 10 // nb_batches:
        printed_percentage = (batch * 10 // nb_batches)
        print(printed_percentage * 10, end='')
    else:
        print('=', end='')
    result_str = [X_train_vocab_rev[c] for c in np.squeeze(result[0,:])]
    if i % 1 == 0 and printed_epoch < i:
        print("\tPrediction str: ", ''.join(result_str[:200]))
    if printed_epoch < i:
        printed_epoch = i

0====10===20===30===40===50====60===70===80===90===1 loss: 4.78812
0	Prediction str:  		ree	re 	e 	e r' re 	e  e  e re  e  '	r'  ' 		  e re r'er'e ' 'e e' re ee  e r' r'er'  ' '' ee  e  e  e re  e  e  e  e ee ee  	  	 ee  e  e    e e eee ''e ee r' 'e' e ee 'eleee e  e  e ee/ e  e  e  e
====10===20===30===40===50====60===70===80===90===2 loss: 3.97392
0	Prediction str:  l   l  	l  c  eee
ee eeeeeee ee ee eee ee>e >   eeeeeee eeeee eee eeeee ee				eeeee ee ee eeeeeeee e e
				eeeeee 
ee eeeeeeeeee eeeeeeeeeeee>eeeeeeeeeeeeeeeeeeeeeeeeeeeee				/	eee eeeeeee	ee	eee eee

====10===20===30===40===50====60===70===80===90===3 loss: 3.79859
0	Prediction str:  titet>/ eee ee e  e        	e	  eeeeee    		 eeeeee
eeeeeeeeeeeee		eeeeeeee/e eeeeeeee eeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeee	 e				eeeeeeeeeeeeteeeeeeeeeee				eeeeeeeeeeeeeTTTTITeIeeeeeeeeIeeeeeeeeeee		
====10===20===30===40===50====60===70===80===90===4 loss: 3.72666
0	Prediction str:  
	e	e	/eeeeetteeee/e/tteeeeeeeteeteeeeeeetetettteteteeee

====10===20===30===40===50====60===70===80===90===30 loss: nan
0	Prediction str:                                                                                                                                                                                                          
====10===20===30===40===50====60===70===80===90===31 loss: nan
0	Prediction str:                                                                                                                                                                                                          
====10===20===30===40===50====60===70===80===90===32 loss: nan
0	Prediction str:                                                                                                                                                                                                          
====10===20===30===40===50====60===70===80===90===33 loss: nan
0	Prediction str:                                                                       

====10===20===30===40===50====60===70===80===90===59 loss: nan
0	Prediction str:                                                                                                                                                                                                          
====10===20===30===40===50====60===70===80===90===60 loss: nan
0	Prediction str:                                                                                                                                                                                                          
====10===20===30===40===50====60===70===80===90===61 loss: nan
0	Prediction str:                                                                                                                                                                                                          
====10===20===30==

KeyboardInterrupt: 

In [12]:
nb_epoches = 1000
loss_hist = []

sess.run(tf.global_variables_initializer())
for i in range(nb_epoches):

    if False:
        in2 = tf.train.range_input_producer(nb_epoches, shuffle=False).dequeue()  # [0, 1, .., epoch_size-1] という整数を順ぐりに無限生成するイテレータ
        x = tf.strided_slice(X_data_one_hot[0], [0, in2 * num_steps],
                             [batch_size, (in2 + 1) * num_steps])  # この使われ方の strided_slice は、data[0:batch_size, i*num_steps:(i+1)*num_steps] だと思って良い
        x.set_shape([batch_size, num_steps])
        y = tf.strided_slice(Y_data[0], [in2 * num_steps],
                             [(in2 + 1) * num_steps])  # 正解 y は x の次に来る単語なので、1を足してスライスを右に一つずらす
        y.set_shape([num_steps])
    else:
        x = X_data_one_hot
        y = Y_data

    l, _ = sess.run([loss, train], feed_dict={X: x, Y: y})
    result = sess.run(prediction, feed_dict={X: x})
    print(i, "loss:", l) #, "prediction: ", result, "true Y: ", Y_data)
    s = sess.run(merged_summary, feed_dict={X: x, Y: y})
    fileWriter.add_summary(s, i)
    if np.isnan(l):
        print("NaN detected. Possibly floating overflow.")
        break
    result_str = [X_train_vocab_rev[c] for c in np.squeeze(result)]
    if i % 10 == 0:
        print("\tPrediction str: ", ''.join(result_str))
    loss_hist.append(l)

0 loss: 4.1273
	Prediction str:  h   eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee e  eeeeeeeeeeeeeeeeeeee e eeeee   eeeeeeeeeeeeeee   e  eee     eeee   eeeeeee eeeeeeeeeee  e  eeeeeeeeeeeeeeeeeee eeeee eeeeeeeeeee  eeeeeeeeeeeeeeee   ee  e eeeeeeeeeeeeeee  e ee eeeeeeeeeeeeee  eee  eeeeeeeeeeeeeeeeee  eeeeee eee eeeee    eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeeeeeeeeeeeeeee  eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
1 loss:

71 loss: 0.471187
72 loss: 0.47262
73 loss: 0.41169
74 loss: 0.395867
75 loss: 0.375893
76 loss: 0.340739
77 loss: 0.321835
78 loss: 0.324801
79 loss: 0.288004
80 loss: 0.271605
	Prediction str:  ** \file  * \brief The main sowsceffile for glsesoplis Windows alinnt. * * The enG y point for mhe ee gr0m, no *atier main ) or WInMain(), res/des.inithis file. * It deals winh the eain tes  lef*oop, OpenGL inidialization, and event mapping. * Some drawing.methods are also dofine  hered */#define USEWIN 1 ///< whethor use pindops alin(wgls)#if !USEWIN#include <wL/glut.h>#else#include "antiglut.h"#include "application.h"#iefine WINVER 0x0500#define WWIN32_WINNT 0x0500#include <windows.h>#eldif#include "Game.h"#include "Giewer.h"#include "Glayer.h"#include "Cntity.h"#include "CoordSys.h"#include "CoordSys.find.h"#include "gqel0ar_file.h"#include "antrodrawih"#include "gtarEnum.h"#include "gmd.h"#include "geybind.h"#include "gotion.h"#include "gls/Glw/ndow.h"#include "gls/GLWmenu.h"#include "Clw/

144 loss: 2.10483
145 loss: 3.21492
146 loss: 3.27899
147 loss: 3.42387
148 loss: 4.58796
149 loss: 4.38127
150 loss: 4.2881
	Prediction str:  *"""fow*iiliiiiy(.i" "#m# ec (udcncdefe nencGlio)eelei"lcnubui(.nucncy(dyiln "um#G.c.nlnody"oncd."#dl ulcul nc cd d.dcd# ecdcdu_dcmuacccl# l_ elbl(nn(dy"ne  nu  d nd.ne luc .n.""eoum# ccd# (.dl nuinl n(nlG. n"cmu_uiieuimuG uuceiueim.d#lllc(.cd"dedo nu_uidum(n r"ie(ndcnluiidne yn(ded"u_u n gdr incd"nyncd"d d#"x n."u "c_ddd n.n(nd.(ndlndu.dudCidqdUn"RId#iRclude " myilu y"""ilu linwlude lClc.c (r.h"1nnclele lxl,l* cc)luc e"i5 ilc ncN"1R C"i5u "ielilc l)idCR)WCN"Rx(xC5L""ilclude " eicbdieug"cRc ninncludrC"C l5  5"illle5eRlRyxl nc 5"iRcmxle lxlulbl_(5"nnclelenlRRclllb  CilCluden"RR,,c,y,e  CiC*lu.b CC)l cey..L ncs ""idMludeL"Ckacuimc ylu  "cnCllxlenlCyeynnecl. "Cilllude dCy.lyRcm#  "llulude "Cmle ""i"llulefixylbxow.h""qlcluUefdx ,.nuo e"nllludenRCle.glu lled. e"qlllu r "xlul5. Cl)ldx #iRcmud# "xlx.5. Cl"c_. g"qduludenlRaRleln 5Cillle5eRlxbll.glCl)cn)

InvalidArgumentError: Nan in summary histogram for: lstm/rnn_outputs
	 [[Node: lstm/rnn_outputs = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](lstm/rnn_outputs/tag, lstm/rnn/transpose/_37)]]

Caused by op 'lstm/rnn_outputs', defined at:
  File "D:\Programs\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Programs\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Programs\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Programs\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "D:\Programs\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "D:\Programs\Anac

In [9]:
saver = tf.train.Saver()

In [ ]:
save_path = saver.save(sess, "TensorFlowLSTM/model.ckpt")
print("Model saved in file: %s" % save_path)

In [10]:
saver.restore(sess, "TensorFlowLSTM/model.ckpt")

INFO:tensorflow:Restoring parameters from TensorFlowLSTM/model.ckpt


In [16]:
result = sess.run(prediction, feed_dict={X: X_data_one_hot})
result_str = [X_train_vocab_rev[c] for c in np.squeeze(result)]
print("\tPrediction str: ", ''.join(result_str))
print("\tTruth str: ", ''.join([X_train_vocab_rev[c] for c in np.squeeze(Y_data)]))

	Prediction str:  ** \file  * \brief The main source file for gltestplus Windows client. * * The entry point for the program, no matter main() or WinMain(), resides in this file. * It deals with the main message loop, OpenGL initialization, and event mapping. * Some drawing methods are also defined here. */#define USEWIN 1 ///< whether use windows api (wgl*)#if !USEWIN#include <GL/glut.h>#else#include "antiglut.h"#include "Application.h"#define WINVER 0x0500#define _WIN32_WINNT 0x0500#include <windows.h>#endif#include "Game.h"#include "Viewer.h"#include "Player.h"#include "Entity.h"#include "CoordSys.h"#include "CoordSys-find.h"#include "stellar_file.h"#include "astrodraw.h"#include "StarEnum.h"#include "cmd.h"#include "keybind.h"#include "motion.h"#include "glw/glwindow.h"#include "glw/GLWmenu.h"#include "glw/GLWchart.h"#include "Docker.h"#include "draw/material.h"//#include "Sceptor.h"#include "glstack.h"#include "Universe.h"#include "glsl.h"#include "sqadapt.h
	Truth str:  ** \file 

In [30]:
result = sess.run(prediction, feed_dict={X: [token_ids_to_one_hot(np.random.randint(max(X_data), size=(sequence_length)), num_classes)]})
for i in range(10):
    result = sess.run(prediction, feed_dict={X: [token_ids_to_one_hot(np.array(result).flat, num_classes)]})
    result_str = [X_train_vocab_rev[c] for c in np.squeeze(result)]
    print("\tPrediction str: ", ''.join(result_str))

	Prediction str:  wliiusondrUSVnnwep hIi  nwete.tatd gloteipmlidee hilLfthmarlieaanrulgh*dnfh pt.lntoipiptiuanoheen hogU_lhflte._e#tweamincapleampiennNNiegignrihmht#iadleihiinl hvrersl#GuarUuaph* hleeesisln .mEhe/tlsairlhta glShehuinghSlzmih#hamgih# > ia3 /iaihfw_NigoihwrhinigigirllS"ehp uln lirpstgiehir phpdlp.hnUplhaaip roeipUdphpltgSa pp/  ls lw/gnenlh ihfpahpshp/ ipo pSfhlaag pgiptein wUaoh/hppi  tthsiph /iSpnhghg.hplh*lur5newi.orelien .l pte /nghlpoaehtieors h    h/ln gvip/tplu p/rgeaeeagaeagpgn p  fpli svplelpigal.i ppghp pvdtt/UrepteghppnmhvhppuflhlofepgUaa"ilggnrrlnolL.irlerssheghnesoslUroanelife lehse shW v g/ireowawv  e/aviialaieehhhlni"/iihn/aee aSp eeepoaapuihlhehlui/lhwinglhliG/hiigppnGgindrfwan sh//. luilhppiedUh /gaeerUd//gpihpappehapU lnphtpossipsnedesppepU  " /glihhphioeh iSlugceipU/   p/ pSpphip.rgSrodigEWIpIeeiaellhowaUac .hti/he pmhgtigie iee p a"Siaileahgihp/  e.plplt/ pgggggirguapEheieoeh#. h#."tse#tntedfh eSieiWn .o u lgetomhihihgnheeehg"p/
	Prediction str:  iiot